In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Иннополис/Итоговый проект/Market Analysis/train_df.csv', index_col=0)
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Иннополис/Итоговый проект/Market Analysis/test_df.csv', index_col=0)

In [3]:
from numpy import loadtxt
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
X=train_df.drop(['reordered','eval_set'],axis=1)

In [5]:
X.head()

,order_id,user_id,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered_count,reordered_sum,reordered_latest,aisle_id,department_id
0,1,112108,4,10,9.0,2067,1,0,NaN,3,19
1,96,17227,6,20,30.0,2067,1,0,NaN,3,19
2,3243,206024,3,15,2.0,2067,1,0,NaN,3,19
3,12950,11456,1,14,13.0,2067,1,0,0.0,3,19
4,17683,177724,5,21,0.0,2067,1,0,NaN,3,19


In [6]:
Y = train_df['reordered']

In [7]:
Y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: reordered, dtype: float64

разделим данные на обучающие и тестовые наборы

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

In [9]:
model = XGBClassifier()

In [10]:
del X, Y

In [11]:
train_y = train_df.reordered.values
out_df = test_df[["order_id", "product_id"]]

In [12]:
train_df = np.array(train_df.drop(["order_id", "user_id", "reordered",'eval_set'], axis=1))
test_df = np.array(test_df.drop(["order_id", "user_id",'eval_set'], axis=1))

Обучиение модели с помощью XGBClassifier

In [13]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [14]:
y_pred = model.predict(X_test)

In [15]:
predictions = [round(value) for value in y_pred]

In [16]:
accuracy = accuracy_score(y_test, predictions)

### Accuracy XGBoost Classifier

In [17]:
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.65%


Построю XGb функцию

In [18]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0):
        params = {}
        params["objective"] = "binary:logistic"
        params['eval_metric'] = 'logloss'
        params["eta"] = 0.05
        params["subsample"] = 0.7
        params["min_child_weight"] = 10
        params["colsample_bytree"] = 0.7
        params["max_depth"] = 8
        params["silent"] = 1
        params["seed"] = seed_val
        num_rounds = 100
        plst = list(params.items())
        xgtrain = xgb.DMatrix(train_X, label=train_y)

        if test_y is not None:
                xgtest = xgb.DMatrix(test_X, label=test_y)
                watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
                model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=10)
        else:
                xgtest = xgb.DMatrix(test_X)
                model = xgb.train(plst, xgtrain, num_rounds)

        pred_test_y = model.predict(xgtest)
        return pred_test_y

Запустим функцию XGB для обучения и предсказания

In [21]:
pred = runXGB(train_df, train_y, test_df)

# использование порогового значения для получения прогнозов
cutoff = 0.2
pred[pred>=cutoff] = 1
pred[pred<cutoff] = 0
out_df["Pred"] = pred
out_df = out_df.loc[out_df["Pred"].astype('int')==1]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:21:46] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [22]:
del train_df, test_df

In [23]:
# если товаров более 1, объедините их в одну строку
def merge_products(x):
    return " ".join(list(x.astype('str')))
out_df = out_df.groupby("order_id")["product_id"].aggregate(merge_products).reset_index()
out_df.columns = ["order_id", "products"]

загрузка результата в выходной файл

In [24]:
# чтение CSV-файла и заполнение продуктов на основе прогнозов
sub_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Иннополис/Итоговый проект/Market Analysis/sample_submission.csv", usecols=["order_id"])
sub_df = pd.merge(sub_df, out_df, how="left", on="order_id")

# когда нет прогнозов, используйте "None"
sub_df["products"].fillna("None", inplace=True)
sub_df.to_csv("xgb_starter_3450.csv", index=False)

In [25]:
X_train['reordered_latest'] = X_train['reordered_latest'].replace(np.nan, 0)
X_test['reordered_latest'] = X_test['reordered_latest'].replace(np.nan, 0)
X_test.head(1000)

,order_id,user_id,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered_count,reordered_sum,reordered_latest,aisle_id,department_id
3052321,2634088,11335,3,18,8.0,6873,1,0,0.0,83,4
5273294,3379345,104957,2,9,30.0,15923,1,0,0.0,67,20
6564246,1344587,174152,0,7,30.0,45692,1,0,0.0,107,19
7705447,2206969,196469,1,9,20.0,31205,1,0,0.0,26,7
6947171,1103780,57331,1,20,15.0,28599,1,0,0.0,130,14
...,...,...,...,...,...,...,...,...,...,...,...
1965286,1099888,79357,5,12,4.0,8277,9,8,0.0,24,4
4473121,256984,62140,0,20,8.0,42959,1,0,0.0,74,17
8158334,355337,70111,3,6,30.0,24341,1,0,0.0,31,7
6278263,1455864,170500,6,14,29.0,11097,1,0,0.0,30,6


# Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
#Logistic Regression model
clf=(LogisticRegression(C=0.02))

In [28]:
#fitting the model
clf.fit(X_train, y_train)

LogisticRegression(C=0.02)

In [29]:
#predictions
pred=clf.predict(X_test)

In [30]:
#accuracy score of Logistic Regression Model
print(accuracy_score(pred, y_test))

0.9019834165224757


# Random Forest

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
clfrf = RandomForestClassifier(max_features="log2", max_depth=11, n_estimators=24,min_samples_split=1000, oob_score=True)

In [33]:
clfrf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(max_depth=11, max_features='log2',
                       min_samples_split=1000, n_estimators=24, oob_score=True)

In [34]:
#predictions
predrf=clfrf.predict(X_test)

In [35]:
#accuracy score for the random forest model
accuracy_score(predrf, y_test)

0.9054271931414816